In [30]:
import numpy
import datetime
import re
import collections
from collections import OrderedDict
import math
import allel
import pandas

In [31]:
# new_file_str = 'binom_call-merged_normal.vcf'
new_vcf_str = 'temp.vcf' 
# new_vcf_str = 'variant_call_resaults/'
ref_vcf_str = '/sbgenomics/project-files/final_project_files/final_merged_pileup.called.vcf'
# ref_vcf_str = 'test_ref.vcf'
output_file_temp_str = 'temp_metrics_files/temp_metrics'

temp_new_vcf_str = 'temp_metrics_files/temp_new.vcf'
temp_ref_vcf_str = 'temp_metrics_files/temp_ref.vcf'


In [353]:
def vcf_to_df(vcf_file):
    callset = allel.read_vcf(vcf_file, fields=['variants/CHROM', 'variants/POS', 'variants/REF', 'variants/ALT',  'calldata/GT'])
    
    df_chrom = pandas.DataFrame.from_dict(callset['variants/CHROM'])#, callset['variants/POS'], callset['variants/REF'])
    df_chrom = df_chrom.rename(columns={0:'CHROM' }) #, 1:'POS', 2:'REF'
    df_pos = pandas.DataFrame.from_dict(callset['variants/POS'], dtype='int64')
    df_pos = df_pos.rename(columns={0:'POS' })
    df_ref = pandas.DataFrame.from_dict(callset['variants/REF'])
    df_ref = df_ref.rename(columns={0:'REF'})
    df_alts = pandas.DataFrame.from_dict(callset['variants/ALT'])
    df_alts = df_alts.rename(columns={0:'ALT_0', 1:'ALT_1'})
    df_gt = pandas.DataFrame.from_dict(callset['calldata/GT'][:,0])
    df_gt = df_gt.rename(columns={0:'GT_0', 1:'GT_1'})
    frames = [ df_chrom, df_pos, df_ref, df_alts, df_gt]
    vcf = pandas.concat(frames, axis=1)
    return vcf

In [346]:
# def declare_variants_only_with_changes(gt0, gt1, ref, alt0, alt1, pos):
#     variants = []
#     for i in range(0,len(gt0)):
# #         print(gt0[i])
# #         print(gt1[i])
# #         print(ref[i])
# #         print(alt0[i])
# #         print(alt1[i])
# #         if(gt0[i] == 0 and gt1[i] == 0):
# #             variants.append(ref[i])
#         if(gt0[i] == 0 and gt1[i] == 1):
#             s = str(pos[i]) + '-' + str(ref[i]) + str(alt0[i])
#             variants.append(s)
# #             variants.append(''.join([ref[i],alt0[i]]))
#         if(gt0[i] == 1 and gt1[i] == 1):
#             s = str(pos[i]) + '-' + alt0[i]
#             variants.append(s)
# #             variants.append(alt0[i])
#         if(gt0[i] == 1 and gt1[i] == 2):
#             s = str(pos[i]) + '-' + str(alt0[i]) + str(alt1[i])
#             variants.append(s)
# #             variants.append(''.join([alt0[i],alt1[i]]))
            
#     return variants

# def declare_variants_without_changes(gt0, gt1, ref, alt0, alt1, pos):
#     variants = []
#     for i in range(0,len(gt0)):
# #         print(gt0[i])
# #         print(gt1[i])
# #         print(ref[i])
# #         print(alt0[i])
# #         print(alt1[i])
#         if(gt0[i] == 0 and gt1[i] == 0):
#             s = str(pos[i]) + '-' + ref[i]
#             variants.append(s)
# #             variants.append(ref[i])
# #         if(gt0[i] == 0 and gt1[i] == 1):
# #             variants.append(''.join([ref[i],alt0[i]]))
# #         if(gt0[i] == 1 and gt1[i] == 1):
# #             variants.append(alt0[i])
# #         if(gt0[i] == 1 and gt1[i] == 2):
# #             variants.append(''.join([alt0[i],alt1[i]]))
            
#     return variants
        
def declare_variants_only_with_changes(gt0, gt1, ref, alt0, alt1, pos):
    variants = []
    for i in range(0,len(gt0)):
    #         print(gt0[i])
    #         print(gt1[i])
    #         print(ref[i])
    #         print(alt0[i])
    #         print(alt1[i])
    #         if(gt0[i] == 0 and gt1[i] == 0):
    #             variants.append(ref[i])
        if(gt0[i] == 0 and gt1[i] == 1):
            s = str(pos) + '-' + str(ref[i]) + str(alt0[i])
            variants.append(s)
    #             variants.append(''.join([ref[i],alt0[i]]))
        if(gt0[i] == 1 and gt1[i] == 1):
            s = str(pos) + '-' + alt0[i]
            variants.append(s)
    #             variants.append(alt0[i])
        if(gt0[i] == 1 and gt1[i] == 2):
            s = str(pos) + '-' + str(alt0[i]) + str(alt1[i])
            variants.append(s)
    #             variants.append(''.join([alt0[i],alt1[i]]))

    return variants

def declare_variants_without_changes(gt0, gt1, ref, alt0, alt1, pos):
    variants = []
    for i in range(0,len(gt0)):
    #         print(gt0[i])
    #         print(gt1[i])
    #         print(ref[i])
    #         print(alt0[i])
    #         print(alt1[i])
        if(gt0[i] == 0 and gt1[i] == 0):
            s = str(pos) + '-' + ref[i]
            variants.append(s)
    #             variants.append(ref[i])
    #         if(gt0[i] == 0 and gt1[i] == 1):
    #             variants.append(''.join([ref[i],alt0[i]]))
    #         if(gt0[i] == 1 and gt1[i] == 1):
    #             variants.append(alt0[i])
    #         if(gt0[i] == 1 and gt1[i] == 2):
    #             variants.append(''.join([alt0[i],alt1[i]]))

    return variants

In [325]:
def calc_basic_metrics(new_dataset, ref_dataset):
    new_position_match = new_dataset['POS'].isin(ref_dataset['POS']).value_counts()
    ref_position_match = ref_dataset['POS'].isin(new_dataset['POS']).value_counts()
    
#     TP = new_position_match[True]
#     FP = new_position_match[False]
#     FN = ref_position_match[False]
    TP = 0
    FP = 0
    FN = 0
    
    if True in new_position_match.index: TP = new_position_match[True]
    if False in new_position_match.index: FP = new_position_match[False]
    if False in ref_position_match.index: FN = ref_position_match[False]
    
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1_score = 2*precision*recall/(precision+recall)
         
    return TP, FP, FN, precision, recall, f1_score

In [355]:
def calc_advance_metrics(new_dataset, ref_dataset):
    new_position_df = new_dataset['POS'].isin(ref_dataset['POS'])
    ref_position_df = ref_dataset['POS'].isin(new_dataset['POS'])
    
    new_position_cnt = new_dataset['POS'].isin(ref_dataset['POS']).value_counts()
    ref_position_cnt = ref_dataset['POS'].isin(new_dataset['POS']).value_counts()
    
    TP = 0
    FP = 0
    FN = 0
    precision = 0
    recall = 0
    f1_score = 0
    
    if False in ref_position_cnt.index: FN += ref_position_cnt[False]
    if False in new_position_cnt.index: FP += new_position_cnt[False]
    
#     print (FP)
#     print (FN)
    
    #indexes of samples from ref file
    ref_match_new_pos = ref_position_df.index[ref_position_df == True].tolist()
#     print (ref_match_new_pos)
    
     #indexes of samples from new file
    new_match_rf_pos = new_position_df.index[new_position_df == True].tolist()
#     print (new_match_rf_pos)
    
    for i in range(0, len(new_match_rf_pos)):

        
        new_gt0 = new_dataset['GT_0'].iloc[new_match_rf_pos].values
        new_gt1 = new_dataset['GT_1'].iloc[new_match_rf_pos].values
        new_ref = new_dataset['REF'].iloc[new_match_rf_pos].values
        new_alt0 = new_dataset['ALT_0'].iloc[new_match_rf_pos].values
        new_alt1 = new_dataset['ALT_1'].iloc[new_match_rf_pos].values
    #     new_pos = new_dataset['POS'].iloc[new_match_rf_pos].values

        new_variants = pandas.Series(declare_variants_only_with_changes(new_gt0, new_gt1, new_ref, new_alt0, new_alt1, 0))
        new_non_variants = pandas.Series(declare_variants_without_changes(new_gt0, new_gt1, new_ref, new_alt0, new_alt1, 0))
        print('new_variants')
        print(new_variants)
        print('new_non_variants')
        print(new_non_variants)

    ref_gt0 = ref_dataset['GT_0'].iloc[ref_match_new_pos].values
    ref_gt1 = ref_dataset['GT_1'].iloc[ref_match_new_pos].values
    ref_ref = ref_dataset['REF'].iloc[ref_match_new_pos].values
    ref_alt0 = ref_dataset['ALT_0'].iloc[ref_match_new_pos].values
    ref_alt1 = ref_dataset['ALT_1'].iloc[ref_match_new_pos].values
#     ref_pos = new_dataset['POS'].iloc[ref_match_new_pos].values
    
    ref_variants = pandas.Series(declare_variants_only_with_changes(ref_gt0, ref_gt1, ref_ref, ref_alt0, ref_alt1, 0))
    ref_non_variants = pandas.Series(declare_variants_without_changes(ref_gt0, ref_gt1, ref_ref, ref_alt0, ref_alt1, 0))
    print('ref_variants')
    print(ref_variants)
    print('ref_non_variants')
    print(ref_non_variants)

    new_call_match_var = new_variants.isin(ref_variants).value_counts()
    new_call_match_non_var = new_non_variants.isin(ref_non_variants).value_counts()
    ref_call_match_var = ref_variants.isin(new_variants).value_counts()
    ref_call_match_non_var = ref_non_variants.isin(new_non_variants).value_counts()

    if True in new_call_match_var.index: TP += new_call_match_var[True]
    if True in new_call_match_non_var.index: TP += new_call_match_non_var[True]
    if False in new_call_match_var.index: FP += new_call_match_var[False]
    if False in new_call_match_non_var.index: FP += new_call_match_non_var[False]
    if False in ref_call_match_var.index: FN += ref_call_match_var[False]
    if False in ref_call_match_non_var.index: FN += ref_call_match_non_var[False]
        
    print (TP)
    print (FP)
    print (FN)
    
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1_score = 2*precision*recall/(precision+recall)
    
    
    return TP, FP, FN, precision, recall, f1_score
    
    

In [327]:
def main(new_file, ref_file):

    new_callset = vcf_to_df(new_file)
    ref_callset = vcf_to_df(ref_file)
    
#     print(new_callset)
#     print(ref_callset)
    
#     print(calc_basic_metrics(new_callset, ref_callset))
#     print(calc_advance_metrics(new_callset, ref_callset))
    
    return calc_advance_metrics(new_callset, ref_callset)


In [356]:

sheet = pandas.DataFrame(columns=['VCF name', 'Binomial probability', 'TP', 'FP', 'FN', 'Precision', 'Recall', 'F1-Score'])
# p = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
p = [0.5]
row = 0
for i in p:
    file = 'variant_call_results/binom_p' + str(i) + '_variant_call.vcf'
    test_file = 'test_new.vcf'
    test_ref = 'test_ref.vcf'
#     TP, FP, FN, prec, rec, f1_s = main(file, ref_vcf_str)
    TP, FP, FN, prec, rec, f1_s = main(test_file, test_ref)
    sheet.loc[row] = [file] + [i, TP, FP, FN, prec, rec, f1_s]
    row += 1
    print('Processing done for ' + file)
sheet.to_csv('metrics.csv', index =False)


[2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16, 17, 18, 19]
new_variants
0     0-A
1    0-AG
dtype: object
new_non_variants
0     0-C
1     0-T
2     0-T
3     0-T
4     0-C
5     0-A
6     0-A
7     0-A
8     0-G
9     0-A
10    0-T
11    0-C
12    0-T
13    0-A
dtype: object
ref_variants
Series([], dtype: float64)
ref_non_variants
0     0-C
1     0-T
2     0-T
3     0-T
4     0-T
5     0-C
6     0-A
7     0-A
8     0-A
9     0-A
10    0-G
11    0-A
12    0-T
13    0-C
14    0-T
15    0-A
dtype: object
14
11
5
Processing done for variant_call_results/binom_p0.5_variant_call.vcf


/opt/conda/lib/python3.6/site-packages/allel/io/vcf_read.py:239: UserWarning: empty CHROM; field: EOL; variant: 23 (:0)
  chunks = [d[0] for d in it]
/opt/conda/lib/python3.6/site-packages/allel/io/vcf_read.py:1516: UserWarning: invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">\n'
  warnings.warn('invalid INFO header: %r' % header)


In [348]:
sheet


,VCF name,Binomial probability,TP,FP,FN,Precision,Recall,F1-Score
0,variant_call_results/binom_p0.5_variant_call.vcf,0.5,2871413,1402,7287,0.999512,0.997469,0.998489


In [93]:
ref_len = ref_callset.shape
new_len = new_callset.shape[0]
ref_len

(80, 8)

In [63]:



with open (new_vcf_str, 'r') as vcf_file, open (ref_vcf_str, 'r') as ref_file, open (temp_new_vcf_str, 'w') as temp_new_vcf, open (temp_ref_vcf_str, 'w') as temp_ref_vcf:
    
    new_callset = allel.read_vcf('test_new.vcf', fields=['variants/CHROM', 'variants/POS', 'variants/REF', 'variants/ALT',  'calldata/GT'])
    ref_callset = allel.read_vcf('test_ref.vcf',  fields=['variants/CHROM', 'variants/POS', 'variants/REF', 'variants/ALT',  'calldata/GT'])
#     print(sorted(callset.keys()))

# with open (temp_new_vcf_str, 'r') as vcf_file, open (temp_ref_vcf_str, 'r') as ref_file, open (output_file_temp_str, 'w') as temp:
#     #TODO
#     # go through both files, collect calls from both positions and compare them
#     #
#     # calculate TP, FP, FN
#     # calculate precision, recall and F-score for those numbers
#     # 
    
    
#     line_num = 0;
#     for line in iter(vcf_file.readline, ''):
#         line_num+=1 
#         if(line[0] != '#'):
#             temp_new_vcf.write(line)
#     for line in iter(ref_file.readline, ''):
#         line_num+=1 
#         if(line[0] != '#'):
#             temp_ref_vcf.write(line)
            
#     print('Pre-processing done.')



/opt/conda/lib/python3.6/site-packages/allel/io/vcf_read.py:1516: UserWarning: invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">\n'
  warnings.warn('invalid INFO header: %r' % header)
